In [ ]:
#default_exp model.ensemble

# Ensemble models

Classes for ensembling several `Learner`s into one model

In [ ]:
# export
from fastai.basics import *
from fastai.tabular.all import *

In [ ]:
#export
class Ensemble():
    def __init__(self, dls, n_models:int=10, learn_func=Learner, cv:bool=False, path=None, 
                 ens_dir='ensemble', **learner_kwargs):
        """Create an ensemble of `Learner`s. learner_func defines what kind of learner is used.
        """
        self.path = Path(path) if path is not None else getattr(dls, 'path', Path('.'))
        self.ens_dir = ens_dir
        self.dls = dls
        self.metrics = learner_kwargs['metrics']
        self.n_models = n_models
        self.models = []
        for i in range(n_models):
            if cv: 
                fold_dls = self.dls # Work in progress
                self.models.append(learn_func(dls=fold_dls, path=path, model_dir=ens_dir, **learner_kwargs))
            else: 
                self.models.append(learn_func(dls=self.dls, path=path, model_dir=ens_dir, **learner_kwargs))
        
    def fit_one_cycle(self, n_iterations, lr_max, **kwargs):
        "Fit the models with fit_one_cycle"
        for m in self.models:
            with m.no_logging(): m.fit_one_cycle(n_iterations, lr_max=lr_max, **kwargs)
    
    
    def validate(self, dl=None) -> pd.DataFrame:
        "Validate all models individually and as an ensemble TODO fix to work with multitarget"
        if dl is None: dl=self.dls[1]
        model_results = torch.cat([m.get_preds(reorder=False, dl=dl)[0][:,:,None] for m in self.models], dim=-1)
        targs = self.models[0].get_preds(reorder=False, dl=dl)[1]
        ensemble_results = model_results.sum(axis=-1) / len(self.models)
        res_ls = []
        for c in range(dl.c):
            res_df = pd.DataFrame(columns=['model_identifier'] + [m.name if hasattr(m, 'name') else m.__name__ for m in self.metrics])
            res_df.loc[0] = (['ensemble'] 
                             + [metric(ensemble_results[:,c], targs[:,c]).item() for metric in self.metrics])
            for i in range(len(self.models)):
                res_df.loc[i+1] = ([i] 
                                   + [metric(model_results[:,c,i], targs[:,c]).item()  for metric in self.metrics])
            res_ls.append(res_df)
        return res_ls
    
    def get_ensemble_preds(self, ds_idx=1, dl=None, with_input=True, with_decoded=False, with_loss=False, act=None,
                           inner=False, reorder=False, cbs=None, **kwargs):
        "get_preds but ensemble results"
        if dl is None: dl=self.dls[1].new(shuffled=False, drop_last=False)
        if reorder and hasattr(dl, 'get_idxs'):
            idxs = dl.get_idxs()
            dl = dl.new(get_idxs = _ConstantFunc(idxs))
        model_results = []
        for m in self.models:
            model_results.append(m.get_preds(dl=dl, with_input=with_input, with_decoded=with_decoded, with_loss=with_loss,
                                             act=act, inner=inner, reorder=reorder, cbs=cbs, **kwargs))
        
        ensemble_results = []
        # iterate through results, could work better:
        obs_idx = 0
        if with_input: 
            ensemble_results.append(model_results[0][obs_idx])
            obs_idx += 1
        ensemble_results.append(sum([res[obs_idx] for res in model_results])/len(self.models))
        obs_idx += 1
        ensemble_results.append(model_results[0][obs_idx])
        obs_idx += 1
        if with_decoded:
            ensemble_results.append(model_results[0][obs_idx])
            obs_idx += 1
        if with_loss:
            ensemble_results.append(sum([res[obs_idx] for res in model_results])/len(self.models))
        return tuple(ensemble_results)
    
    def predict(self, item):
        model_results = [m.predict(item) for m in self.models]
        ensemble_results = sum([res[-1] for res in model_results])/len(self.models)
        ensemble_dec_results = sum([res[-2] for res in model_results])/len(self.models)
        return (model_results[0][0], ensemble_dec_results, ensemble_results)

    

NameError: name 'Learner' is not defined

Create `Ensemble` of `Learners` with `n_models`. Works pretty much like a single `Learner`

In [ ]:
#export
@patch
def export(self:Ensemble, folder='ensemble', pickle_protocol=2):
    """Export `Ensemble` and models to self.path/folder, with `Ensemble` related attributes in `ensemble.pkl`
    and each model in `model_i.pkl`"""
    old_dls = self.dls
    self.dls = self.dls.new_empty()
    old_models = self.models
    self.models = []
    if not os.path.exists(self.path/folder): os.makedirs(self.path/folder)
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        torch.save(self, self.path/folder/'ensemble.pkl', pickle_protocol=pickle_protocol)
    for i, m in enumerate(old_models):
        m.export(fname=self.path/folder/f'model_{i}.pkl', pickle_protocol=pickle_protocol)
    self.dls = old_dls
    self.models = old_models

In [ ]:
# export
def load_ensemble(folder, cpu=True):
    "Load `Ensemble` of learners at once for inference"
    res = torch.load(f'{folder}/ensemble.pkl', map_location='cpu' if cpu else None)
    for i in range(res.n_models):
        res.models.append(load_learner(f'{folder}/model_{i}.pkl', cpu=cpu))
    return res